# Calculer des fréquences

Qu’il soit sobre (*Small Language Model*), volumineux (*Large Language Model*) ou prévu pour un domaine particulier (*Specialized Language Model*), un modèle de langage est une représentation statistique de la distribution des symboles d’une langue (lettres, phonèmes, mots…) en vue d’effectuer des prédictions. En se basant par exemple sur la fréquence d’apparition des lettres dans un corpus, il est tout à fait envisageable de générer du texte. Ou plutôt une collection de lettres. Des outils supplémentaires peuvent être intégrés au modèle, comme un lexique, ou une distribution de la longueur des mots et des phrases dans un texte.

La limite principale d’un modèle de langage est qu’il est incapable de rendre compte d’une langue, tout au plus de son état à un certain moment donné en vue des paramètres initiaux qui lui ont été fournis. Il repose fondamentalement sur un corpus, qui lui-même est une extraction, et reproduira ses biais. Si votre corpus contient *La montagne de l’âme* de Gao Xingjian, il risque de donner trop d’importance aux formes conjuguées des verbes à la deuxième personne du singulier. Et si vous incluez l’ensemble du contenu d’Internet, qui lui-même est de plus en plus alimenté par des IA génératives, vous ne manquerez pas de véhiculer ses fictions.

## Définitions

**Occurrence :** Une occurrence désigne la survenue d’un événement dans le temps ou dans l’espace. En linguistique, le terme fait référence à l’apparition d’un mot dans une phrase, d’une lettre dans un mot, d’un morphème, ou d’un trait spécifique. Au pluriel, il est utilisé pour compter le nombre d’apparitions d’un phénomène donné.

**Nombre d’apparitions :** Le nombre d’apparitions correspond au décompte exact d’un événement, autrement dit, à l’effectif total des occurrences observées.

**Fréquence :** La notion est souvent liée à celle d’occurrence et peut être exprimée de deux manières :

  - **La fréquence absolue :** nombre total d’occurrences d’un événement dans un corpus (p. ex. : la fréquence d’occurrences de la lettre *e* dans le verbe *vole* est de 1).
  - **La fréquence relative :** Rapport entre la fréquence absolue d’un événement et l’effectif total de l’ensemble. Elle se situe dans l’intervalle $[0,1]$ et peut être exprimée en pourcentage (p. ex. : la fréquence relative de la lettre e dans vole est $\frac{1}{4}$, soit 0,25 ou 25 %).

**Distribution de fréquence :** Répartition des fréquences d’occurrences d’un ou plusieurs phénomènes dans un corpus, souvent visualisée sous forme de tableaux, graphiques ou histogrammes.

**Mot-forme :** Représentation graphique d’un mot tel qu’il apparaît dans un texte ou un corpus, sans distinction de variations grammaticales (p. ex. : *mange*, *manges* et *mangerons* sont des mots-formes différents).

**Lemme :** Forme canonique d’un mot utilisée comme entrée principale dans un dictionnaire ou un lexique (p. ex. : *manger* est le lemme des mots-formes *mange*, *manges* et *mangerons*).

**Type :** En linguistique et en analyse textuelle, un type désigne une unité lexicale unique, abstraite, considérée sans prendre en compte ses occurrences spécifiques dans un texte. Par exemple, dans la phrase « Le chat mange le poisson », les types sont *le*, *chat*, *mange* et *poisson*.

**Token :** Un token désigne chaque occurrence concrète d’un type dans un texte. On dénombre par exemple cinq tokens dans la phrase « Le chat mange le poisson ».

## Aperçu de la difficulté de la tâche

### Compter des occurrences

L’opération revient à subdiviser un objet en éléments et à compter le nombre de fois où chaque élément apparaît. Prenons une liste d’entiers entre 0 et 10 et calculons le nombre de fois où apparaît le nombre 9 :

In [ ]:
! echo "5 8 1 3 6 7 9 9 7 3" | grep -o 9 | wc -l

Où :

- L’option `-o` de `grep` permet d’extraire chaque occurrence du motif sur une ligne distincte
- La commande `wc -l` compte le nombre de lignes.

On peut réaliser une opération similaire en comptabilisant les apparitions de la lettre *e* dans une phrase :

In [ ]:
! echo "En pratique, un pêcheur pêche avec une canne." \
| grep -o e | wc -l

L’informatique dissocie par défaut le caractère *e* de ses versions accentuée *ê* et majuscule *E*. Il est heureusement possible de jouer sur le motif transmis à la commande `grep` pour améliorer le résultat :

In [ ]:
! echo "En pratique, un pêcheur pêche avec une canne." \
| grep -o "[eêE]" | wc -l

Une version encore simplifiée fait appel à l’option `-i` pour rendre l’expression rationnelle insensible à la casse :

In [ ]:
! echo "En pratique, un pêcheur pêche avec une canne." \
| grep -oi "[eê]" | wc -l

### Normaliser avec la translittération

Le cas précédent nécessite encore de faire l’inventaire de toutes les manières de représenter la lettre *e* en français afin d’être sûr de n’en manquer aucune. Une technique plus simple repose sur la **translittération**, une procédure qui consiste à transcrire un texte d’un alphabet (ou système d’écriture) vers un autre tout en respectant au mieux les sons et la signification des mots.

En règle générale, il existe plusieurs systèmes de translittérations. Prenons l’exemple de la conversion de l’arabe vers le latin qui peut s’effectuer en respectant la norme ISO 233 ou la norme ALA-LC :

أنتِ التي تحدّثين، أوّل الكلامِ.

Cette phrase issue des *Mille et une nuits* se transcrit, en simplifiant légèrement :

- أنتِ : "Anti" ;
- التي : "Allatī" ;
- تحدّثين : "Tuhaddithīna" ;
- أوّل : "Awwal" ;
- الكلامِ : "Al-Kalām".

À quoi cela peut-il nous aider pour notre phrase remplie de diacritiques ? Et si on appliquait le principe de la translittération pour convertir d’un jeu de caractères vers un autre ? Quel jeu de caractères serait un bon candidat pour l’alphabet latin non accentué ?

Essayons de transcrire notre phrase de l’UTF-8 vers l’ASCII :

In [ ]:
! echo "En pratique, un pêcheur pêche avec une canne." \
| iconv -f UTF-8 -t ASCII//TRANSLIT 2>/dev/null

Où :

- `-t ASCII//TRANSLIT` force la transcription vers l’ASCII selon un dictionnaire de translittération ;
- `2>/dev/null` redirige les erreurs vers un périphérique spécial qui fait office de trou noir.

Il reste à appliquer le filtre et à compter le nombre de lignes de résultats :

In [ ]:
! echo "En pratique, un pêcheur pêche avec une canne." \
| iconv -f UTF-8 -t ASCII//TRANSLIT 2>/dev/null \
| grep -io "e" | wc -l

#### Cas limite : exemple de la lettre *i* en turc"

Le turc utilise quatre variantes de *i* :

- *i* (minuscule avec point)
- *ı* (minuscule sans point)
- *İ* (majuscule avec point)
- *I* (majuscule sans point)

Une translittération de l’UTF-8 vers l’ASCII confond la lettre minuscule latine i sans point avec le chiffre *1* :

In [ ]:
! echo "İstanbul'da iki kişi ile ılık bir çay içtim." \
| iconv -f UTF-8 -t ASCII//TRANSLIT 2> /dev/null

Dans ce cas, la seule alternative consiste à effectuer un pré-traitement :

In [ ]:
! echo "İstanbul'da iki kişi ile ılık bir çay içtim." \
| tr 'ı' 'i' \
| iconv -f UTF-8 -t ASCII//TRANSLIT 2> /dev/null \
| grep -oi "i" | wc -l

### Les classes de caractères Unicode

Avec les langues européennes, la complexité de la tâche reste contenue. Qu’en est-il avec, par exemple, des langues asiatiques ?

Lorsqu’il s’agit de compter uniquement des correspondances particulières, aucune difficulté supplémentaire à signaler :

In [ ]:
! echo "今日ははじめて、はっきりとわかった。" \
| grep -o は | wc -l

Le japonais est composé de plusieurs classes de caractères : les kanji, les katakana et les hiragana. Si l’objectif devient de comptabiliser tous les caractères d’une classe, il convient de faire appel à une option `-P` pour étendre les expressions rationnelles aux classes Unicode :

In [ ]:
! echo "今日ははじめて、はっきりとわかった。" \
| grep -oP "\p{Script=Han}" | wc -l

L’instruction précédente a repéré les caractères *今* et *日*, issus du script Han (Ensemble de caractères logographiques utilisés dans plusieurs systèmes d’écriture de langues asiatiques.). Notons que la syntaxe `\p{Han}` peut, dans certains cas, conserver également les signes de ponctuation *、* et *。*. La recherche inclut alors tous les caractères définis comme Han, sans distinction de la catégorie "script".

**Remarque :** La version BSD `grep` installée par défaut sur macOS ne supporte pas l’option `-P` pour les expressions compatibles Perl (PCRE). Il faut alors installer GNU `grep` et utiliser ensuite la commande `ggrep`.

Ce comportement ouvre la possibilité de traiter des textes multilingues, comme par exemple repérer les citations en grec ou en latin dans un texte de Nietzsche.

En plus des lettres d’un alphabet spécifique, on dénombre plusieurs classes Unicode :

- `\p{L}` : Toutes les lettres peu importe l’alphabet.
- `\p{N}` : Toutes les représentations de chiffres.
- `\p{P}` : Tous les symboles de ponctuation.
- `\p{S}` : Les symboles (mathématiques, monétaires et autres).
- `\p{Z}` : Tous les caractères d’espacement, incluant la tabulation, les sauts de ligne etc.
- `\p{C}` : Les caractères dits "de contrôle" comme les retours chariot, les sauts de ligne, etc.) et autres caractères invisibles.
- `\p{M}` : Les marques de diacritiques qui s’ajoutent aux lettres.
- `\p{Cased_Letter}` : Toutes les lettres majuscules et minuscules.
- `\p{Lower}` : Les lettre minuscules.
- `\p{Upper}` : Les lettres majuscules.

**Remarque :** Sur macOS, les caractères Unicode sont généralement stockés sous la forme précomposée NFC (*Normalization Form Composed*), où les lettres accentuées et autres caractères combinés sont représentés sous une seule unité plutôt qu’en plusieurs composants distincts (ex: *é* au lieu de *e* + *´*). Pour cette raison, certaines expressions utilisant les classes Unicode ne répondront pas forcément de la manière attendue.

## Travailler avec des fréquences

Les fréquences d’occurrences sont une première entrée dans la linguistique quantitative. Que l’on dénombre les mots-formes, les types, les lemmes, les étiquettes ou toute autre combinaison, elles servent à fournir certaines indications chiffrées sur un texte ou un corpus.

### Fréquences cumulées

La fréquence cumulée correspond à la somme des fréquences depuis le mot le plus fréquent jusqu’au rang considéré. Elle permet d’observer des phénomènes remarquables, comme le fait qu’un petit nombre de mots représente souvent une grande partie du texte. Dans la plupart des langues, les 100 mots les plus fréquents peuvent représenter jusqu’à 50 % des occurrences d’un texte, principalement en raison des mots grammaticaux (articles, prépositions, etc.).

Sans analyse grammaticale, un tableau de la fréquence d’occurences de la phrase "Le pêcheur pêche avec une canne à pêche" donne :

|Type|Fréquence|Fréquence cumulée absolue|Fréquence cumulée relative|
|-|-|-|-|
|pêche|2|2|0.250|
|avec|1|3|0.375|
|à|1|4|0.5|
|canne|1|5|0.625|
|le|1|6|0.750|
|pêcheur|1|7|0.875|
|un|1|8|1|

### Poids relatif des mots

Le poids d’un mot-forme dans un texte peut être calculé de plusieurs manières :

- Par sa fréquence relative (nombre d’occurrences divisé par le nombre total de mots).
- Par son rang dans la liste des mots les plus fréquents.
- Par une mesure de pondération comme son TF-IDF (*Term Frequency-Inverse Document Frequency*) qui prend en compte à la fois sa fréquence dans le document et sa rareté dans un corpus de référence.

Ces mesures permettent de repérer les mots caractéristiques d’un texte, d’un auteur ou d’un genre particulier.

### Mots-clés, mots vides et listes d’exclusion

Dans un corpus, on ne voudra jamais analyser tous les mots. Soit notre attention se portera sur certains mots-clés à conserver, soit, au contraire, nous souhaiterons exclure certains résultats du calcul de fréquences. Parmi ces derniers, une catégorie particulière est réservée aux mots vides de sens (*stopwords* en anglais), qui sont des mots très fréquents dans une langue et qui, même s’ils remplissent une fonction grammaticale essentielle, apportent peu d’information sur le contenu sémantique d’un texte. Il s’agit principalement des déterminants, des prépositions, des pronoms et autres mots outils.

L’identification et le filtrage de ces mots est une étape cruciale en analyse textuelle pour plusieurs raisons :

- Ils peuvent représenter jusqu’à 50 % des occurrences dans un texte ;
- ils faussent les statistiques de fréquence brute ;
- ils créent du bruit dans l’analyse thématique ou la recherche d’information.

Pour exclure un fichier contenant une liste de mots avant le comptage des fréquences, il convient d’intégrer un filtrage à la chaîne de traitement :

In [ ]:
! grep -vwFf ./files/stopwords.txt

Où :

- `-v` (*invert match*) sert à exclure les résultats de l’expression rationnelle à suivre ;
- `-w` assure que le moteur de filtre recherche des mots complets ;
- `-F` contraint à effectuer une recherche sur des chaînes littérales ;
- `-f` précise que ces chaînes proviennent d’un fichier.

**Remarque :** Comme la commande `grep` agit sur une ligne, un filtrage trop précoce éliminera des pans entiers du texte à analyser si un seul des mots de la liste d’exclusion est détecté, ce qui sera probablement le cas pour les mots vides. Une bonne pratique consiste à tokeniser au préalable le texte en mettant un mot par ligne.

En post-traitement, la commande sera appelée pour épurer la liste des fréquences :

In [ ]:
# exclude some words
! grep -vwFf ./files/stopwords.txt ./files/frequencies.tsv

D’autres fois, pourtant, l’objet de recherche se concentrera sur l’emploi de ces mots grammaticaux. C’est le cas notamment dans les études qui impliquent une analyse stylistique : le style d’un·e auteur·rice se révèle souvent dans sa façon d’articuler et de composer les syntagmes entre eux.

In [ ]:
# keywords only
! grep -wFf ./files/keywords.txt ./files/frequencies.tsv

### Rapport Type-Token (TTR)

Le *Type-Token Ratio* (TTR) est un indicateur utilisé en linguistique quantitative pour évaluer la diversité lexicale d’un corpus. Il s’agit du rapport entre le nombre de types et le nombre de tokens dans un texte donné. Il s’exprime mathématiquement par la formule :

$$
\text{TTR} = \frac{\text{Nombre de types}}{\text{Nombre de tokens}}
$$

Un TTR élevé indique une grande diversité lexicale, ce qui est souvent le cas pour des textes littéraires riches ou des discours techniques variés. À l’inverse, un TTR faible peut refléter une répétition importante des mêmes mots, comme dans des textes didactiques ou des dialogues informels.

#### Exemple

Prenons le texte suivant :

>"Le chat mange le poisson et le chien aboie."

**Tokens (9) :** "Le", "chat", "mange", "le", "poisson", "et", "le", "chien", "aboie".

**Types (7) :** "le", "chat", "mange", "poisson", "et", "chien", "aboie".

$$
TTR = 7 \div 9 \approx 0,78
$$

#### Traitement des mots grammaticaux

En règle générale, les mots grammaticaux sont exclus du calcul du TTR afin de se concentrer sur les mots porteurs de sens. Cette exclusion améliore la pertinence du ratio obtenu comme indicateur de la diversité lexicale. Si l’objectif en revanche est d’évaluer la richesse syntaxique, de comparer des registres de langue ou d’étudier des phénomènes spécifiques, il conviendra sans doute de les garder.

#### Limites du TTR

Le TTR est sensible à la taille du corpus : plus un texte est long, plus il y a de chances que de nouveaux tokens soient des répétitions de types existants. Pour pallier cette limitation, il peut être utile de calculer le *Mean Segmental Type-Token Ratio* (MSTTR) qui consiste à diviser un texte en parties égales et à établir la moyenne des TTR des différentes parties, comme donné par la formule :

$$
\text{MSTTR} = \frac{\sum_{i=1}^n \text{TTR}_i}{n}
$$

La commande `split` permet de découper un fichier sur le critère du nombre de lignes :

```sh
split -l 1000 -d {file.txt} chunk_
```

Où :

- `-l` impose de créer des fichiers de *n* lignes ;
- `-d` force l’emploi de suffixes numériques dans la dénomination des fichiers ;
- `chunk_` indique le préfixe à utiliser pour nommer les fichiers de sortie.

### Distribution des longueurs de mots

L’analyse de la longueur des mots offre un autre angle d’approche statistique. Elle permet de :

- Caractériser le style d'un auteur (préférence pour les mots courts ou longs).
- Comparer différents genres textuels (les textes techniques utilisent généralement des mots plus longs).
- Identifier des motifs linguistiques propres à une langue (en allemand, les mots composés augmentent la moyenne).

On peut représenter cette distribution sous forme d’histogramme ou calculer des indicateurs comme la longueur moyenne des mots, la médiane, ou l’écart-type.

### Lois de distribution

Les fréquences lexicales suivent souvent des lois statistiques remarquables, notamment :

- **La loi de Zipf :** la fréquence d’un mot est inversement proportionnelle à son rang.
- **La loi de Mandelbrot :** une version affinée de la loi de Zipf qui tient compte de la structure hiérarchique du lexique.
- **La loi de Heap :** relation entre la taille du vocabulaire et la taille du texte.

Ces lois permettent de modéliser et de prédire le comportement des fréquences lexicales dans les textes en langage naturel.